# Descriptive statistics of the Database

This notebook presents descriptive statistics of the samples in the database by analyzing the metadata that accompanies the audio files.

The metadata is available in the `data/metadata.csv` file.


In [1]:
import os

import pandas as pd

In [2]:
# Load samples metadata
metadata = pd.read_csv(
    os.path.join(os.getcwd(), "data", "metadata.csv"), index_col="index"
)
metadata.head()

,sex,age_yrs,record_point,pathology,pathology_state,breath_record_type,breath_record_source,record_quality
index,,,,,,,,
0,M,4,Second intercostal space,Relapsing Obstructive Bronchitis,Unspecified Disease Control,Calm,Phone,Average
1,M,47,Second intercostal space,Bronchial asthma,Exacerbation or/and Uncontrolled,Calm,Phone,Average
2,F,14,Second intercostal space,Healthy,Unspecified Disease Control,Calm,Phone,Average
3,F,8,Chest from behind,Bronchial asthma,Partially controlled,Calm,Adobe Audition,Average
4,M,37,Second intercostal space,Healthy,Exacerbation or/and Uncontrolled,Calm,Phone,Good


In [3]:
print(f"Overall the database has {len(metadata.index)} samples")

Overall the database has 1371 samples


In [4]:
def combine_ill(grouped_stats: pd.DataFrame) -> pd.DataFrame:
    """Combine "Ill(not asthma)" samples into one column."""
    data = [
        grouped_stats["Bronchial asthma"],
        grouped_stats["Healthy"],
        grouped_stats["Relapsing Obstructive Bronchitis"]
        + grouped_stats["Pneumonia"]
        + grouped_stats["Chronic Obstructive Pulmonary Disease"]
        + grouped_stats["Bronchopulmonary dysplasia"]
        + grouped_stats["Cystic fibrosis"]
        + grouped_stats["Blind"],
    ]

    headers = ["Asthma", "Healthy", "Ill(not asthma)"]
    stats = pd.concat(data, axis=1, keys=headers)
    stats["Total"] = stats.sum(axis=1)
    for column in ["Asthma", "Healthy", "Ill(not asthma)", "Total"]:
        stats.loc["Total", column] = stats[column].sum()
        stats[column] = stats[column].astype(int)
    return stats

## 1. Generalized sex distribution.

Group `Ill(not asthma)` samples together

In [5]:
grouped_stats = (
    metadata.groupby(["pathology", "sex"])
    .size()
    .unstack("pathology")
    .fillna(0)
    .astype(int)
)
stats = combine_ill(grouped_stats)
stats

,Asthma,Healthy,Ill(not asthma),Total
sex,,,,
F,313,52,24,389
M,800,81,101,982
Total,1113,133,125,1371


## 2. Detailed sex distribution.

In [6]:
# Group database diagnoses by sex
raw_stats = (
    metadata.groupby(["pathology", "sex"]).size().unstack("sex").fillna(0).astype(int)
)

# Exclude asthma asthme because we handle it separately
raw_stats.drop("Bronchial asthma", inplace=True)

# Move the "Healthy" row up
idx = [raw_stats.index.get_loc("Healthy")] + [
    i for i in range(len(raw_stats)) if i != raw_stats.index.get_loc("Healthy")
]
raw_stats = raw_stats.iloc[idx]

# Query the metadata to get just the asthma samples to build sex distribution
asthma_only_stats = metadata[metadata["pathology"] == "Bronchial asthma"]
asthma_only_stats = (
    asthma_only_stats.groupby(["pathology_state", "sex"])
    .size()
    .unstack("sex")
    .fillna(0)
    .astype(int)
)

# Transpose the dataframe to sum up and rename different conditions
stats_transposed = asthma_only_stats.T
stats_transposed.rename(
    columns={
        "Exacerbation or/and Uncontrolled": "Asthma (Exacerbation or/and Uncontrolled)",
        "Well-controlled": "Asthma (Well-controlled)",
        "Partially controlled": "Asthma (Partially controlled)",
    },
    inplace=True,
)

stats_transposed["Asthma (Other)"] = stats_transposed["Unspecified Disease Control"]
stats_transposed.drop("Unspecified Disease Control", inplace=True, axis=1)

asthma_stats = stats_transposed.T
asthma_stats.reindex(
    [
        "Asthma (Exacerbation or/and Uncontrolled)",
        "Asthma (Well-controlled)",
        "Asthma (Partially controlled)",
        "Asthma (Other)",
    ]
)
asthma_stats.index.name = "pathology"

# Sum and format the result
raw_stats = pd.concat(objs=[asthma_stats, raw_stats])
raw_stats["Total"] = raw_stats.sum(axis=1)
raw_stats.loc["Total", "F"] = raw_stats["F"].sum()
raw_stats.loc["Total", "M"] = raw_stats["M"].sum()
raw_stats.loc["Total", "Total"] = raw_stats["Total"].sum()
raw_stats = raw_stats.astype(int)
raw_stats

sex,F,M,Total
pathology,,,
Asthma (Exacerbation or/and Uncontrolled),53,131,184
Asthma (Partially controlled),113,307,420
Asthma (Well-controlled),77,166,243
Asthma (Other),70,196,266
Healthy,52,81,133
Blind,0,15,15
Bronchopulmonary dysplasia,1,0,1
Chronic Obstructive Pulmonary Disease,0,1,1
Cystic fibrosis,0,1,1


## 3. Diagnosis and recording site distribution.

In [7]:
grouped_stats = (
    metadata.groupby(["record_point", "pathology"])
    .size()
    .unstack("pathology")
    .fillna(0)
    .astype(int)
)

stats = combine_ill(grouped_stats)
stats

,Asthma,Healthy,Ill(not asthma),Total
record_point,,,,
Chest from behind,285,9,10,304
Oral cavity,256,0,12,268
Second intercostal space,563,123,101,787
Trachea,9,1,2,12
Total,1113,133,125,1371


## 4. Age distribution

In [8]:
bins = [-1000, 2, 4, 13, 20, 110]
labels = ["0-2", "2-4", "4-13", "13-20", "20+"]
metadata["age_group"] = pd.cut(
    metadata["age_yrs"], bins=bins, labels=labels, right=False
)

grouped_stats = (
    metadata.groupby(["age_group", "pathology"])
    .size()
    .unstack("pathology")
    .fillna(0)
    .astype(int)
)

stats = combine_ill(grouped_stats)
stats = stats.T
stats

age_group,0-2,2-4,4-13,13-20,20+,Total
Asthma,14,8,515,516,60,1113
Healthy,6,0,32,13,82,133
Ill(not asthma),1,6,110,8,0,125
Total,21,14,657,537,142,1371
